In [11]:
import pandas as pd
import csv

def remove_decimal_zero(value):
    # Chuyển giá trị thành chuỗi
    str_value = str(value)

    # Kiểm tra xem giá trị có phải là số nguyên không
    if '.' in str_value:
        # Loại bỏ số 0 sau dấu thập phân nếu chúng là 0
        str_value = str_value.rstrip('0').rstrip('.')

    return str_value

def format_qty(value):
    return int(value) if value % 1 == 0 else value

def progress_raw_excel(excel_file_path):
    df = pd.read_excel(excel_file_path)
    print(df)
    
    # Convert 'level' column to integers only when it contains data
    df['level'] = pd.to_numeric(df['level'], errors='coerce')
    print(df)
    #print(df['level'].dtype)
    dataAfterConvert = df
    
    levelBefore = 0
    
    key_value = 0;
    
    my_object = {
        'Key0':[],
        'SubKey0':[],
    }
    
    sub_index = 1
    
    
    for index, row in dataAfterConvert.iterrows():
        
        level_value = int(row['level'])
        
        #print(f"Index: {index}, Level:{level_value}, dataType: {type(level_value)}")
        if level_value != levelBefore:
            sub_index = 1
            #level_value different than levelBefore will save to object
            if level_value != levelBefore + 1:
                my_object[f"Key{key_value}"].append(remove_decimal_zero(row['PN']))
                key_value +=1
                my_object[f"Key{key_value}"]=[]
                my_object[f"SubKey{key_value}"]=[]
                
            elif level_value == levelBefore + 1:
                my_object[f"SubKey{key_value-1}"].append({'index':sub_index, 'QTY':remove_decimal_zero(row['QTY']), 'PN':remove_decimal_zero(row['PN'])})
                sub_index +=1
                
                
        elif level_value == levelBefore:
            my_object[f"SubKey{key_value-1}"].append({'index':sub_index, 'QTY':remove_decimal_zero(row['QTY']), 'PN':remove_decimal_zero(row['PN'])})
            sub_index +=1
          
        levelBefore = level_value

        
    #print(my_object)  
    return my_object



def create_csv(header_content, data_rows):
        # Specify the header with meaningful column labels
    header = [header_content, "", ""]

    # Specify the additional data row
    data_row = ["IT", "QTY", "PART NO."]

    # Specify the additional data rows
#     data_rows = [
#         ["1", "2", "32460719"],
#         ["2", "2", "32460720"],
#         ["3", "3", "32460622"],
#         ["4", "4", "32460629"],
#         ["5", "5", "32980004"],
#         ["6", "6", "32980005"],
#         ["7", "7", "40002253"],
#         ["8", "8", "32880071"],
#         ["9", "9", "32740018"],
#         ["10", "3.5", "40021116"],
#         ["11", "1", "3214026912"],
#         ["12", "1", "32140141"],
#         ["13", "0.08", "32500214"]
#     ]
    data_rows = data_rows

    # Get the first element of the header for the filename
    file_name = f"{header[0]}.csv"

    # Open the file in write mode
    with open(file_name, mode='w', newline='') as csv_file:
        # Create a CSV writer object
        csv_writer = csv.writer(csv_file)

        # Write the header to the CSV file
        csv_writer.writerow(header)

        # Write the data row to the CSV file
        csv_writer.writerow(data_row)

        # Write the data rows to the CSV file
        csv_writer.writerows(data_rows)

    print(f"CSV file '{file_name}' created successfully.")
    
def create_header_and_dataRows(data):

    result = []

    datarows = []
    headers = []

    current_key = None

    for key, value in data.items():
        if key.startswith('Key') and value and value[0]:  # Check if the list is not empty and the first element is not empty
            current_key = value[0]
            headers.append(current_key)
    for key, value in data.items():
        if key.startswith('SubKey') and value and value[0]:
            datarow = [] 
            for item in value:
                print(item)
                datarow.append([item['index'],item['QTY'],item['PN']])
                #for subkey, subvalue in item.items():
                    #print(item.items())
                    #datarow.append(subvalue)
            datarows.append(datarow)

    #print("Headers:", headers)
    print("Data Rows:", datarows)

    result = {'header': headers, 'datarows': datarows}
    #print(result)
    return result

# Đường dẫn đến file Excel
excel_file_path = '//tsclient/D/temp/IC35/103.xlsx'

# Đường dẫn đến file CSV kết quả
result_csv_path = 'output.csv'

# Gọi chương trình create_csv
result_progress_raw_excel = progress_raw_excel(excel_file_path)
#print(result_progress_raw_excel)
result_header_and_datarow = create_header_and_dataRows(result_progress_raw_excel)
print(result_header_and_datarow)
print(len(result_header_and_datarow['header']))

for i in range(len(result_header_and_datarow['header'])):
    header_value = result_header_and_datarow['header'][i]
    datarow = result_header_and_datarow['datarows'][i]
    print(header_value)
    print(datarow)
    create_csv(header_value, datarow)
   

     level    QTY        PN      Supplier part  \
0        5   1.00  40001729                NaN   
1        6   2.00  32460719            AT04-4P   
2        6   2.00  32460720               W4-P   
3        6   1.00  32460622            DT06-4S   
4        6   1.00  32460629               AW4S   
..     ...    ...       ...                ...   
407      5   1.00  32980989             RVL2-5   
408      5   9.00  32740018          114017-ZZ   
409      5  11.00  32140198  XSD-GXL-14AWG-BLU   
410      5   1.78  32140263                NaN   
411      5   0.02  32500214                NaN   

                                                 DESC.  QTY.1  UOM  MJV INV.  \
0                               HARNESS, DRI ABOVE ROT   1.00  EA.       NaN   
1                    CONNECTOR, DT04-4P, 4 PIN DEUTSCH   2.00  EA.       NaN   
2                 WEDGELOCK, 4 PIN FOR RECEPTACLE, W4P   2.00  EA.       NaN   
3    CONNECTOR, HOUSING 4-PIN,DEUTSCH PLUG, #16 SOC...   1.00  EA.       NaN   